In [1]:
import sqlite3 as sql
import pandas as pd
import numpy as np

In [2]:
db_conn = sql.connect('new_sql_db')

In [3]:
execute_sql = db_conn.cursor()

In [4]:
details=pd.read_csv("C:/Users/gehna/Data Science/Projects/Data/Details.csv")
orders=pd.read_csv("C:/Users/gehna/Data Science/Projects/Data/Orders.csv")

In [5]:
details.to_sql("Details", con = db_conn, if_exists = 'replace')

1500

In [6]:
orders.to_sql("Orders", con = db_conn, if_exists = 'replace')

500

In [7]:
a = pd.read_sql_query('SELECT * from Details', con = db_conn)
a

,index,OrderID,Amount,Profit,Quantity,Category,SubCategory,PaymentMode
0,0,B-25681,1096,658,7,Electronics,Electronic Games,COD
1,1,B-26055,5729,64,14,Furniture,Chairs,EMI
2,2,B-25955,2927,146,8,Furniture,Bookcases,EMI
3,3,B-26093,2847,712,8,Electronics,Printers,Credit Card
4,4,B-25602,2617,1151,4,Electronics,Phones,Credit Card
...,...,...,...,...,...,...,...,...
1495,1495,B-25700,7,-3,2,Clothing,Hankerchief,COD
1496,1496,B-25757,3151,-35,7,Clothing,Trousers,EMI
1497,1497,B-25973,4141,1698,13,Electronics,Printers,COD
1498,1498,B-25698,7,-2,1,Clothing,Hankerchief,COD


In [8]:
b = pd.read_sql_query('select * from Orders', con = db_conn)

In [9]:
b

,index,OrderID,OrderDate,CustomerName,State,City
0,0,B-26055,10-03-2018,Harivansh,Uttar Pradesh,Mathura
1,1,B-25993,03-02-2018,Madhav,Delhi,Delhi
2,2,B-25973,24-01-2018,Madan Mohan,Uttar Pradesh,Mathura
3,3,B-25923,27-12-2018,Gopal,Maharashtra,Mumbai
4,4,B-25757,21-08-2018,Vishakha,Madhya Pradesh,Indore
...,...,...,...,...,...,...
495,495,B-25742,03-08-2018,Ashwin,Goa,Goa
496,496,B-26088,26-03-2018,Bhavna,Sikkim,Gangtok
497,497,B-25707,01-07-2018,Shivani,Maharashtra,Mumbai
498,498,B-25758,22-08-2018,Shubham,Himachal Pradesh,Simla


# Question - 1
Which category have the highest profit?

In [10]:
q1 = pd.read_sql_query('select sum(profit), category from Details group by category limit 1', con = db_conn)
q1

,sum(profit),Category
0,13325,Clothing


# Question - 2
Which payment mode has used in more transactions?

In [11]:
q2 = pd.read_sql_query('select count(PaymentMode) as cnt, PaymentMode from Details group by PaymentMode order by cnt desc', con = db_conn)
q2

,cnt,PaymentMode
0,684,COD
1,331,UPI
2,202,Debit Card
3,163,Credit Card
4,120,EMI


# Question - 3
What are the top 3 values of profit?

In [12]:
q3 = pd.read_sql_query('select * from Details order by Profit desc limit 3', con = db_conn)
q3

,index,OrderID,Amount,Profit,Quantity,Category,SubCategory,PaymentMode
0,59,B-25608,1364,1864,5,Furniture,Tables,Credit Card
1,1497,B-25973,4141,1698,13,Electronics,Printers,COD
2,32,B-26022,1824,1303,8,Electronics,Phones,Credit Card


# Question - 4
How many statewise customers are there?

In [13]:
q4 = pd.read_sql_query('select State, count(State) from Orders group by State order by count(State) desc', con = db_conn)
q4

,State,count(State)
0,Maharashtra,94
1,Madhya Pradesh,93
2,Rajasthan,32
3,Gujarat,27
4,Uttar Pradesh,25
5,Punjab,25
6,Delhi,24
7,West Bengal,22
8,Karnataka,20
9,Kerala,16


# Question - 5
Which city has the best customers? We would like to give a promotional coupouns in the city we made the most money. Write a query that returns one city that has the highest sum of invoice totals. Return both the city name and sum of amount.

In [14]:
 merged_df = pd.merge(details, orders, how='inner', on='OrderID')

q5 = merged_df.groupby('City')['Profit'].sum()
group = q5.reset_index().sort_values(by='Profit', ascending=False)
group.head(1)

,City,Profit
10,Indore,6763


In [15]:
merged_df['TotalExp'] = merged_df['Amount'] + merged_df['Profit']

# Question - 6
Who is the best customer? The customer who has spent the most money will be declared the best customer. Write a query that returns the person who has spent the most money. 

In [16]:
q6 = merged_df.groupby('CustomerName')['TotalExp'].sum()
grouped_sum_df = q6.reset_index().sort_values(by='TotalExp', ascending=False)
grouped_sum_df.head(1)

,CustomerName,TotalExp
142,Madhav,11213


# Question - 7
Write a query to return name, state of all customers. Return your list ordered alphabetically by name starting with A.

In [17]:
q7 = pd.read_sql_query('select CustomerName, State from Orders order by CustomerName asc', con = db_conn)
q7

,CustomerName,State
0,Aakanksha,Madhya Pradesh
1,Aarushi,Tamil Nadu
2,Aarushi,Tamil Nadu
3,Aarushi,Tamil Nadu
4,Aastha,Himachal Pradesh
...,...,...
495,Yogesh,Maharashtra
496,Yogesh,Bihar
497,Yogesh,Bihar
498,Yogesh,Bihar


# Question - 8
Categorize the products on the basis of subcategories to check sales.

In [18]:
q8 = pd.read_sql_query('select Category, SubCategory, count(SubCategory)as TotalCntEachCat from Details group by SubCategory order by Category', con = db_conn)
q8

,Category,SubCategory,TotalCntEachCat
0,Clothing,Hankerchief,197
1,Clothing,Kurti,47
2,Clothing,Leggings,53
3,Clothing,Saree,211
4,Clothing,Shirt,69
5,Clothing,Skirt,64
6,Clothing,Stole,192
7,Clothing,T-shirt,77
8,Clothing,Trousers,39
9,Electronics,Accessories,72


# Question - 9
How much quantity is purchased in each categories?

In [19]:
q9 = pd.read_sql_query('select sum(Quantity), SubCategory, Category from Details group by SubCategory order by Category', con = db_conn)
q9

,sum(Quantity),SubCategory,Category
0,741,Hankerchief,Clothing
1,164,Kurti,Clothing
2,186,Leggings,Clothing
3,795,Saree,Clothing
4,271,Shirt,Clothing
5,248,Skirt,Clothing
6,671,Stole,Clothing
7,305,T-shirt,Clothing
8,135,Trousers,Clothing
9,262,Accessories,Electronics


# Question - 10
Find how much amount spent by each customer on products by their state. Write a query to return customer name, state and total spent.

In [20]:
q10 = merged_df.groupby(['CustomerName', 'State'])['Amount'].sum().reset_index().sort_values(by='Amount',ascending = False)
q10

,CustomerName,State,Amount
124,Harivansh,Uttar Pradesh,9902
168,Madan Mohan,Uttar Pradesh,7766
306,Shiva,Maharashtra,6339
169,Madhav,Delhi,6026
166,Lalita,Uttar Pradesh,5809
...,...,...,...
130,Hemangi,Delhi,13
104,Dinesh,Tamil Nadu,12
72,Ashwin,Goa,11
311,Shivani,Maharashtra,8


# Question - 11
We want to find most popular product for each state. We determine the most popular product as the product with the highest amount of purchases. Write a query that returns each state along with the top product. For States where the maximum number of purchases is shared return all products.

In [27]:
merged_df = pd.merge(details, orders, how='inner', on='OrderID')
merged_df['Product'] = merged_df['Category'] + ' - ' + merged_df['SubCategory']

grouped_sum = merged_df.groupby(['State', 'Product'])['Quantity'].sum().reset_index()
most_popular_products = grouped_sum.loc[grouped_sum.groupby('State')['Quantity'].idxmax()].reset_index(drop=True)

max_amounts = grouped_sum.groupby('State')['Quantity'].transform(max)
popular_products_with_ties = grouped_sum[grouped_sum['Quantity'] == max_amounts].reset_index(drop=True)

print("Most popular products:")
print(most_popular_products)

print("\nPopular products with ties:")
print(popular_products_with_ties) 

Most popular products:
                State                  Product  Quantity
0      Andhra Pradesh   Clothing - Hankerchief        32
1               Bihar   Clothing - Hankerchief        26
2               Delhi         Clothing - Stole        55
3                 Goa   Clothing - Hankerchief        39
4             Gujarat         Clothing - Saree        50
5             Haryana  Furniture - Furnishings        20
6    Himachal Pradesh         Clothing - Stole        33
7   Jammu and Kashmir         Clothing - Saree        33
8           Karnataka         Clothing - Shirt        30
9             Kerala          Clothing - Saree        40
10     Madhya Pradesh   Clothing - Hankerchief       206
11        Maharashtra         Clothing - Saree       173
12           Nagaland   Clothing - Hankerchief        25
13             Punjab         Clothing - Saree        43
14          Rajasthan         Clothing - Saree        50
15             Sikkim   Clothing - Hankerchief        29
16      

# Question - 12
Write a query that determines the customer that has spent the most on product for each state. Write a query that returns the state along with the top customer and how much they spent. For States where the top amount spent is shared, provide all customers who spent this amount.

In [22]:
merged_df = pd.merge(details, orders, how='inner', on='OrderID')

merged_df['total_spent'] = (merged_df['Amount'] + merged_df['Profit']) * merged_df['Quantity']
grouped_sum = merged_df.groupby(['State', 'CustomerName'])['total_spent'].sum().reset_index()
most_spent = grouped_sum.loc[grouped_sum.groupby('State')['total_spent'].idxmax()].reset_index(drop=True)
max_amounts = grouped_sum.groupby('State')['total_spent'].transform(max)
top_spenders_with_ties = grouped_sum[grouped_sum['total_spent'] == max_amounts].reset_index(drop=True)

print("Top spenders for each state:")
print(most_spent)

print("\nTop spenders with ties:")
print(top_spenders_with_ties)

Top spenders for each state:
                State CustomerName  total_spent
0      Andhra Pradesh      Krutika        20959
1               Bihar       Yogesh        16491
2               Delhi       Madhav        30657
3                 Goa     Abhishek        17974
4             Gujarat     Shreyshi        21155
5             Haryana       Mukesh        21503
6    Himachal Pradesh       Aastha        17303
7   Jammu and Kashmir        Kirti        17592
8           Karnataka         Ajay         9859
9             Kerala       Shourya        14336
10     Madhya Pradesh     Abhijeet        28045
11        Maharashtra        Shiva        47323
12           Nagaland        Farah        27164
13             Punjab    Shrichand        32453
14          Rajasthan      Paridhi        32664
15             Sikkim         Amit         7338
16         Tamil Nadu      Aarushi        32616
17      Uttar Pradesh    Harivansh       102264
18        West Bengal      Parishi        27196

Top spende